### Import stuff

In [ ]:
!pip install radis

from radis.db.classes import get_molecule_identifier
from radis.levels.partfunc import PartFuncHAPI

from radis.io.hitemp import fetch_hitemp
from radis.db.classes import get_molecule
from radis.phys.constants import hc_k

import numpy as np
from numpy import exp, pi

### Function to reduce the memory usage of pandas dataframe

In [9]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

### Fetch Databank

In [12]:
df0 = fetch_hitemp(molecule='CH4', databank_name='HITEMP-CH4', isotope='1, 2, 3', load_wavenum_min=2000, load_wavenum_max=4000, local_databases='~/')

Using existing database HITEMP-CH4


### Let's Crunch

In [13]:
df1, NAlist = reduce_mem_usage(df0)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 565.2839660644531  MB
******************************
Column:  id
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  iso
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  wav
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  int
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  A
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  airbrd
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  selbrd
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  El
dtype before:  float64
dtype after:  float32
***************